## Deep Learning Model - CNN Multi-Label Text Classification
### GLOVE with Sub Themes (Right Now Ran for Main Themes)

In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using plaidml.keras.backend backend.


### Loading Data

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

### Creating a Unified Dataframe for CNN Ready Model

In [3]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,:12]], axis = 1)

In [4]:
df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data_df = df

In [6]:
data_df.shape

(10376, 13)

### Pre-processing

#### General

In [7]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [9]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in data_df['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
data_df['preprocessed_comments']=preprocessed_synopsis

In [10]:
data_df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH,preprocessed_comments
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0,real diversity you need create seats table mea...
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0,keep building warmer provide warm water bathroom
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0,better communication top
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0,would beneficial management not micro manage
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0,education applicable job


#### Our Pre-processor

In [ ]:
def preprocess_comments(text, 
                        min_token_len = 2, 
                        irrelevant_pos = ['PRON', 'SPACE', 'PUNCT', 'ADV', 
                                          'ADP', 'CCONJ', 'AUX', 'PRP'],
                        avoid_entities = ['PERSON', 'ORG', 'LOC', 'GPE']):
# note: Didn't use the following options in the `preprocess_comments`...
#    - 'PROPN' because it erases proper names as 'George', but also words as orange.
#    - 'DET' since it removes the word 'no', which changes the meaning of a sentence.
# *for more information see link: https://universaldependencies.org/u/pos/
    """
    Given text, min_token_len, irrelevant_pos and avoid_entities, carries out 
    preprocessing of the text and returns list of preprocessed text. 
    
    Parameters
    -------------
    text : (list) 
        the list of text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    avoid_entities : (list)
        a list of entity labels to be avoided
    
    Returns
    -------------
    (list) list of preprocessed text
    
    Example
    -------------
    >>> example = ["Hello, I'm George and I love swimming!",
                   "I am a really good cook; what about you?",
                   "Contact me at george23@gmail.com"]

    >>> preprocess(example)
    (output:) ['hello love swimming', 'good cook', 'contact']
    """

    result = []
    
    others = ["'s", "the", "that", "this", "to", "-PRON-"]
    # I add "-PRON-" that erase "my", "your", etc. other way to erase them is to
    #   use adding 'DET' to irrelevant_pos but it would erase the word 'no' too.
    
    for sent in text:
        sent = sent.lower()
        sent = re.sub(r"facebook", "social media", sent)
        sent = re.sub(r"twitter", "social media", sent)
        sent = re.sub(r"instagram", "social media", sent)
        sent = re.sub(r"whatsapp", "social media", sent)
        sent = re.sub(r"linkedin", "social media", sent)
        sent = re.sub(r"snapchat", "social media", sent)
        
        result_sent = []
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
        # This helps to detect names of persons, organization and dates
        
        for token in doc:            
            if (token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities or
                str(token.lemma_) in others or
                len(token) < min_token_len):
                continue
            else:
                result_sent.append(token.lemma_)
        result.append(" ".join(result_sent))
    return result

In [ ]:
data_df['preprocessed_comments']=preprocess_comments(data_df['Comment'])

In [ ]:
data_df.head()

In [ ]:
data_df.shape

### Splitting into Train and Test

In [11]:
data_df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH,preprocessed_comments
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0,real diversity you need create seats table mea...
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0,keep building warmer provide warm water bathroom
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0,better communication top
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0,would beneficial management not micro manage
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0,education applicable job


In [12]:
X_train = data_df[['preprocessed_comments']]
y_train = data_df.drop(['Comment', 'preprocessed_comments'], axis=1)

### Preparing Labels

**Max length of sentence**

In [13]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(data_df['Comment'].apply(max_len))
max_len

150

### Vocab Size

**For General Use**

In [14]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

11919


**Glove**

In [30]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/karan/Downloads/glove/glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [31]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in vect.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [32]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.071531  ,  0.29840001, -0.2476    , ..., -0.057444  ,
        -0.040008  , -0.051519  ],
       [ 0.0083903 ,  0.28769001, -0.23466   , ..., -0.66409999,
         0.10303   ,  0.1219    ],
       ...,
       [-0.60427999,  0.26354   ,  0.031186  , ..., -0.15360001,
        -0.35222   ,  0.10186   ],
       [-0.23841999,  0.054833  ,  0.40522   , ..., -0.13877   ,
         0.09218   , -0.031022  ],
       [-0.31097001,  0.66358   ,  0.10347   , ..., -0.040046  ,
         0.75432998,  0.34955999]])

## Modelling CNN

#### Padding to make all sequences of same length

**Training Data**

In [33]:
encoded_docs_train = vect.texts_to_sequences(X_train['preprocessed_comments'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[  504   585    36 ...     0     0     0]
 [  134    54  3393 ...     0     0     0]
 [    7    29   234 ...     0     0     0]
 ...
 [  697     4    12 ...     0     0     0]
 [  476  2745 11917 ...     0     0     0]
 [ 1147   593   791 ...     0     0     0]]


In [34]:
padded_docs_train.shape

(10376, 150)

#### Defining Model

## Glove

In [35]:
y_train = np.array(y_train)

In [36]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 3
embed_size = 300 # for glove we are using 100d dataset
n_class = 12

In [37]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))

model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          3575700   
_________________________________________________________________
dropout_3 (Dropout)          (None, 150, 300)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 148, 250)          225250    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 72, 250)           187750    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 36, 250)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9000)              0         
__________

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [39]:
# Train Model
model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.15)

Train on 8819 samples, validate on 1557 samples
Epoch 1/3
8819/8819 [==============================] - 2675s 303ms/step - loss: 0.3617 - acc: 0.8805 - val_loss: 0.2893 - val_acc: 0.8986
Epoch 2/3
8819/8819 [==============================] - 2838s 322ms/step - loss: 0.2745 - acc: 0.9024 - val_loss: 0.2302 - val_acc: 0.9165
Epoch 3/3
8819/8819 [==============================] - 2797s 317ms/step - loss: 0.2249 - acc: 0.9183 - val_loss: 0.2060 - val_acc: 0.9249


**Evaluation on Validation data**

In [40]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y
X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)
# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')


score = model.evaluate(padded_docs_valid,y_valid)
score

2594/2594 [==============================] - 590s 227ms/step


[0.20097736289341264, 0.9246659160303721]

## Saving Model

In [ ]:
model.save('../models/cnn_glove_sub_themes.pkl')

## Precision & Recall

In [28]:
pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

2594/2594 [==============================] - 234s 90ms/step


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

For threshold:  0.1
Micro-average quality numbers
Precision: 0.3492, Recall: 0.8361, F1-measure: 0.4927


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5233, Recall: 0.6791, F1-measure: 0.5911


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6490, Recall: 0.5641, F1-measure: 0.6036


For threshold:  0.4
Micro-average quality numbers
Precision: 0.7375, Recall: 0.4729, F1-measure: 0.5763


For threshold:  0.5
Micro-average quality numbers
Precision: 0.8057, Recall: 0.3955, F1-measure: 0.5306


For threshold:  0.6
Micro-average quality numbers
Precision: 0.8609, Recall: 0.3386, F1-measure: 0.4860


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8975, Recall: 0.2830, F1-measure: 0.4303


For threshold:  0.8
Micro-average quality numbers
Precision: 0.9281, Recall: 0.2211, F1-measure: 0.3571


For threshold:  0.9
Micro-average quality numbers
Precision: 0.9570, Recall: 0.1476, F1-measure: 0.2557




## For 300d Model

In [41]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y
X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)
# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')


score = model.evaluate(padded_docs_valid,y_valid)
score

2594/2594 [==============================] - 591s 228ms/step


[0.20097736289341264, 0.9246659160303721]

In [42]:
pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

2594/2594 [==============================] - 828s 319ms/step


In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

For threshold:  0.1
Micro-average quality numbers
Precision: 0.4024, Recall: 0.8256, F1-measure: 0.5411


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5757, Recall: 0.6960, F1-measure: 0.6301


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6917, Recall: 0.5959, F1-measure: 0.6402


For threshold:  0.4
Micro-average quality numbers
Precision: 0.7718, Recall: 0.5133, F1-measure: 0.6165


For threshold:  0.5
Micro-average quality numbers
Precision: 0.8269, Recall: 0.4450, F1-measure: 0.5786


For threshold:  0.6
Micro-average quality numbers
Precision: 0.8701, Recall: 0.3889, F1-measure: 0.5375


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8982, Recall: 0.3317, F1-measure: 0.4845


For threshold:  0.8
Micro-average quality numbers
Precision: 0.9351, Recall: 0.2670, F1-measure: 0.4154


For threshold:  0.9
Micro-average quality numbers
Precision: 0.9658, Recall: 0.1954, F1-measure: 0.3251


